<a href="https://colab.research.google.com/github/dieguito1331/cienciaDatos/blob/master/UCEMA/02 - KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# KNN — Demo paso a paso (interactiva)

**Objetivo:** explicar visualmente el algoritmo *k*-Nearest Neighbors mostrando **un paso a la vez**:
1. Datos
2. Punto de prueba
3. Círculo hasta el vecino *k*-ésimo
4. Líneas hacia los *k* vecinos
5. Vecinos resaltados y **predicción** (voto/probabilidades)
6. **Frontera de decisión** (opcional) sobre una grilla


In [9]:
# ==============================
# KNN interactivo: punto nuevo + vecinos + sensibilidad vs k (FIX nombres)
# ==============================
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from ipywidgets import interact, FloatSlider, IntSlider, VBox, HBox, HTML
from IPython.display import display

# --- (Opcional) dataset de ejemplo ---
try:
    X, y
except NameError:
    from sklearn.datasets import make_blobs
    X, labels = make_blobs(
        n_samples=20000,
        centers=[[-4, -2], [-2, 1], [-5, 2],   # clase 0
                 [ 3,  2], [ 5,-1], [ 2,-3]], # clase 1
        cluster_std=2.2,
        random_state=42
    )
    y = (labels >= 3).astype(int)

# Aseguramos forma correcta de etiquetas
y_labels = np.asarray(y).ravel()

# Rangos sliders
x_lo, x_hi = X[:,0].min()*1.1, X[:,0].max()*1.1
y_lo, y_hi = X[:,1].min()*1.1, X[:,1].max()*1.1

pred_colors = {0: "#3b82f6", 1: "#ef4444"}  # azul/rojo

def knn_predict_at_point(px=0.0, py=0.0, k=5, kmax=31):
    test_point = np.array([px, py], dtype=float)

    # -------- Plot principal --------
    fig, ax = plt.subplots(figsize=(6.7, 6.2))

    # Datos base (usar y_labels, NO el parámetro del slider)
    ax.scatter(
        X[:,0], X[:,1],
        c=y_labels, cmap="coolwarm",
        alpha=0.45, s=12, edgecolor="none"
    )

    # Modelo KNN (k actual)
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X, y_labels)
    pred = int(knn.predict(test_point.reshape(1, -1))[0])
    proba = knn.predict_proba(test_point.reshape(1, -1))[0]

    # Punto de prueba
    ax.scatter(
        [test_point[0]], [test_point[1]],
        color=pred_colors[pred], s=90, marker="x",
        label=f"Punto nuevo (pred={pred})"
    )

    # Vecinos y círculo
    dists = np.sqrt(((X - test_point)**2).sum(axis=1))
    idx_sorted = np.argsort(dists)
    nn_idx = idx_sorted[:k]
    kth_radius = dists[idx_sorted[k-1]]

    circ = plt.Circle((test_point[0], test_point[1]), kth_radius,
                      fill=False, linestyle="--", color="black", linewidth=1.5, alpha=0.9)
    ax.add_patch(circ)

    ax.scatter(
        X[nn_idx,0], X[nn_idx,1],
        facecolors="none", edgecolors="black",
        s=140, linewidths=1.5, label=f"{k} vecinos"
    )

    # Líneas a los más cercanos (solo 200 para no ensuciar)
    show_n = min(200, len(X))
    for i in idx_sorted[:show_n]:
        ax.plot([test_point[0], X[i,0]], [test_point[1], X[i,1]],
                color="gray", alpha=0.05, linewidth=0.4)

    ax.set_xlim(x_lo, x_hi); ax.set_ylim(y_lo, y_hi)
    ax.set_xlabel("Feature 1"); ax.set_ylabel("Feature 2")
    ax.grid(True, linestyle="--", alpha=0.25)
    ax.legend(loc="upper right")
    ax.set_title(
        f"KNN en (x={px:.1f}, y={py:.1f}) | k={k} → clase={pred} | prob={proba.round(3)}",
        fontsize=11, pad=12
    )
    plt.show()

    # -------- Sensibilidad vs k --------
    ks = np.arange(1, kmax+1, 2)
    probs = []
    for kk in ks:
        m = KNeighborsClassifier(n_neighbors=kk).fit(X, y_labels)
        probs.append(m.predict_proba(test_point.reshape(1, -1))[0][1])
    probs = np.array(probs)

    plt.figure(figsize=(6.7, 3.3))
    plt.plot(ks, probs, marker="o")
    plt.axvline(k, linestyle="--")
    plt.title("Sensibilidad: Probabilidad de clase 1 según k")
    plt.xlabel("k (número de vecinos)"); plt.ylabel("P(clase=1)")
    plt.xticks(ks); plt.grid(True, linestyle="--", alpha=0.3)
    plt.show()

    # Resumen
    display(HTML(f"""
    <b>Resumen</b><br>
    Punto: ({px:.2f}, {py:.2f})<br>
    k = {k} → Predicción = <b>{pred}</b> | Probabilidades = {proba.round(3).tolist()}<br>
    Radio al vecino k-ésimo = {kth_radius:.3f}
    """))

# -------- UI (renombrado px/py para no chocar con y_labels) --------
ui = VBox([
    HBox([
        FloatSlider(value=0.0, min=x_lo, max=x_hi, step=(x_hi-x_lo)/200, description='px', readout_format='.1f'),
        FloatSlider(value=0.0, min=y_lo, max=y_hi, step=(y_hi-y_lo)/200, description='py', readout_format='.1f'),
    ]),
    HBox([
        IntSlider(value=5, min=1, max=49, step=2, description='k'),
        IntSlider(value=31, min=5, max=99, step=2, description='kmax'),
    ])
])

@interact
def _view(px=ui.children[0].children[0], py=ui.children[0].children[1],
          k=ui.children[1].children[0], kmax=ui.children[1].children[1]):
    knn_predict_at_point(px, py, k, kmax)




interactive(children=(FloatSlider(value=0.0, description='px', max=415.5616679239506, min=-433.6640276780177, …